<a href="https://colab.research.google.com/github/suhyeon03/study_ML_kaggle/blob/main/kaggle_evaluation_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>